#  Multi-Agent Orchestration with NVIDIA NeMo Agent Toolkit

In this notebook, we showcase how the NeMo Agent Toolkit can be used to use a mixture of inbuilt tools and agents, as well as custom tools and workflows. Multi-agent orchestration is an important concept in many agentic AI facets: notably runtime and token efficiency. Here we aim to show how simple multi-agent orchestration is able to be implemented when using NeMo Agent Toolkit. We show how an orchestration agent can call tools and sub-agents to facilitate complex tasks.


# Table of Contents

- [0.0) Setup](#setup)
  - [0.1) Prerequisites](#prereqs)
  - [0.2) API Keys](#api-keys)
  - [0.3) Data Sources](#data-sources)
  - [0.4) Installing NeMo Agent Toolkit](#installing-nat)
- [1.0) Defining Tools for the Multi-Agent Workflow](#creating-workflow)
  - [1.1) Total Product Sales Tool](#product-sales-tool)
  - [1.2) Sales Per Day Tool](#sales-per-day-tool)
  - [1.3) Detect Outliers Tool](#detect-outliers-tool)
  - [1.4) Technical Specs Retrieval Tool](#technical-specs-tool)
  - [1.5) Data Analysis/Plotting Tools](#plotting-tools)
  - [1.6) Register The Tools](#register-tools)
- [2.0) Adding an Agent Orchestrator](#adding-orchestrator)
  - [2.1) Agent Orchestrator Workflow Configuration File](#orchestrator-config)
  - [2.2) Running the Workflow](#running-orchestrator-workflow)
- [3.0) Adding a Custom Agent](#adding-custom-agent)
  - [3.1) Human-in-the-Loop (HITL) Approval Tool](#hitl-tool)
  - [3.2) Graph Summarizer Tool](#graph-summarizer-tool)
  - [3.3) Custom Data Visualization Agent With HITL Approval](#custom-viz-agent)
  - [3.4) Custom Agent Workflow Configuration File](#custom-agent-config)
  - [3.5) Running the Workflow](#running-custom-workflow)
- [4.0) Next Steps](#next-steps)

<span style="color:rgb(0, 31, 153); font-style: italic;">Note: In Google Colab use the Table of Contents tab to navigate.</span>


By the conclusion of this example, we will create a simple mixture-of-agents that serves as an assistant in retail sales.

> **Note:** _This is just an example agent system that uses dummy data. The intention is to demonstrate some of the capabilities of this toolkit and how a new user can get familiar with it._

This agent system has:

1. A **supervisor** agent that routes incoming requests to the downstream agent expert
2. A **data insight agent** that is a tool-calling agent capable of answering questions about sales data
3. A **RAG agent** that is capable of answering questions about products using context from a product catalog
4. A **data visualization agent** that is capable of plotting graphs and trends

We demonstrate the following capabilities:
* RAG
* Multi-framework support
* Human-in-the-Loop
* Multi-agent support


<a id="setup"></a>
# 0.0) Setup


<a id="prereqs"></a>
## 0.1) Prerequisites

Before users run this notebook, it is encouraged that they run `3_adding_tools_to_agents.py`. As this notebook will assume that users are familiar with tool calling agents, and gloss over the details of the initial agent that has tool calling setup.

- **Platform:** Linux, macOS, or Windows
- **Python:** version 3.11, 3.12, or 3.13
- **Python Packages:** `pip`

<a id="api-keys"></a>
## 0.2) API Keys

For this notebook, you will need the following API keys to run all examples end-to-end:

- **NVIDIA Build:** You can obtain an NVIDIA Build API Key by creating an [NVIDIA Build](https://build.nvidia.com) account and generating a key at https://build.nvidia.com/settings/api-keys

Then you can run the cell below:

In [ ]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

<a id="data-sources"></a>
## 0.3) Data Sources

Several data files are required for this example. To keep this as a stand-alone example, the files are included here as cells which can be run to create them.

The following cell creates the `data` directory as well as a `rag` subdirectory

In [ ]:
!mkdir -p data/rag

The following cell writes the `data/retail_sales_data.csv` file.

In [ ]:
%%writefile data/retail_sales_data.csv
Date,StoreID,Product,UnitsSold,Revenue,Promotion
2024-01-01,S001,Laptop,1,1000,No
2024-01-01,S001,Phone,9,4500,No
2024-01-01,S001,Tablet,2,600,No
2024-01-01,S002,Laptop,9,9000,No
2024-01-01,S002,Phone,10,5000,No
2024-01-01,S002,Tablet,5,1500,No
2024-01-02,S001,Laptop,4,4000,No
2024-01-02,S001,Phone,11,5500,No
2024-01-02,S001,Tablet,7,2100,No
2024-01-02,S002,Laptop,7,7000,No
2024-01-02,S002,Phone,6,3000,No
2024-01-02,S002,Tablet,9,2700,No
2024-01-03,S001,Laptop,6,6000,No
2024-01-03,S001,Phone,7,3500,No
2024-01-03,S001,Tablet,8,2400,No
2024-01-03,S002,Laptop,3,3000,No
2024-01-03,S002,Phone,16,8000,No
2024-01-03,S002,Tablet,5,1500,No
2024-01-04,S001,Laptop,5,5000,No
2024-01-04,S001,Phone,11,5500,No
2024-01-04,S001,Tablet,9,2700,No
2024-01-04,S002,Laptop,2,2000,No
2024-01-04,S002,Phone,12,6000,No
2024-01-04,S002,Tablet,7,2100,No
2024-01-05,S001,Laptop,8,8000,No
2024-01-05,S001,Phone,18,9000,No
2024-01-05,S001,Tablet,5,1500,No
2024-01-05,S002,Laptop,7,7000,No
2024-01-05,S002,Phone,10,5000,No
2024-01-05,S002,Tablet,10,3000,No
2024-01-06,S001,Laptop,9,9000,No
2024-01-06,S001,Phone,11,5500,No
2024-01-06,S001,Tablet,5,1500,No
2024-01-06,S002,Laptop,5,5000,No
2024-01-06,S002,Phone,14,7000,No
2024-01-06,S002,Tablet,10,3000,No
2024-01-07,S001,Laptop,2,2000,No
2024-01-07,S001,Phone,15,7500,No
2024-01-07,S001,Tablet,6,1800,No
2024-01-07,S002,Laptop,0,0,No
2024-01-07,S002,Phone,7,3500,No
2024-01-07,S002,Tablet,12,3600,No
2024-01-08,S001,Laptop,5,5000,No
2024-01-08,S001,Phone,8,4000,No
2024-01-08,S001,Tablet,5,1500,No
2024-01-08,S002,Laptop,4,4000,No
2024-01-08,S002,Phone,11,5500,No
2024-01-08,S002,Tablet,9,2700,No
2024-01-09,S001,Laptop,6,6000,No
2024-01-09,S001,Phone,9,4500,No
2024-01-09,S001,Tablet,8,2400,No
2024-01-09,S002,Laptop,7,7000,No
2024-01-09,S002,Phone,11,5500,No
2024-01-09,S002,Tablet,8,2400,No
2024-01-10,S001,Laptop,6,6000,No
2024-01-10,S001,Phone,11,5500,No
2024-01-10,S001,Tablet,5,1500,No
2024-01-10,S002,Laptop,8,8000,No
2024-01-10,S002,Phone,5,2500,No
2024-01-10,S002,Tablet,6,1800,No
2024-01-11,S001,Laptop,5,5000,No
2024-01-11,S001,Phone,7,3500,No
2024-01-11,S001,Tablet,5,1500,No
2024-01-11,S002,Laptop,4,4000,No
2024-01-11,S002,Phone,10,5000,No
2024-01-11,S002,Tablet,4,1200,No
2024-01-12,S001,Laptop,2,2000,No
2024-01-12,S001,Phone,10,5000,No
2024-01-12,S001,Tablet,9,2700,No
2024-01-12,S002,Laptop,8,8000,No
2024-01-12,S002,Phone,10,5000,No
2024-01-12,S002,Tablet,14,4200,No
2024-01-13,S001,Laptop,3,3000,No
2024-01-13,S001,Phone,6,3000,No
2024-01-13,S001,Tablet,9,2700,No
2024-01-13,S002,Laptop,1,1000,No
2024-01-13,S002,Phone,12,6000,No
2024-01-13,S002,Tablet,7,2100,No
2024-01-14,S001,Laptop,4,4000,Yes
2024-01-14,S001,Phone,16,8000,Yes
2024-01-14,S001,Tablet,4,1200,Yes
2024-01-14,S002,Laptop,5,5000,Yes
2024-01-14,S002,Phone,14,7000,Yes
2024-01-14,S002,Tablet,6,1800,Yes
2024-01-15,S001,Laptop,9,9000,No
2024-01-15,S001,Phone,6,3000,No
2024-01-15,S001,Tablet,11,3300,No
2024-01-15,S002,Laptop,5,5000,No
2024-01-15,S002,Phone,10,5000,No
2024-01-15,S002,Tablet,4,1200,No
2024-01-16,S001,Laptop,6,6000,No
2024-01-16,S001,Phone,11,5500,No
2024-01-16,S001,Tablet,5,1500,No
2024-01-16,S002,Laptop,4,4000,No
2024-01-16,S002,Phone,7,3500,No
2024-01-16,S002,Tablet,4,1200,No
2024-01-17,S001,Laptop,6,6000,No
2024-01-17,S001,Phone,14,7000,No
2024-01-17,S001,Tablet,7,2100,No
2024-01-17,S002,Laptop,3,3000,No
2024-01-17,S002,Phone,7,3500,No
2024-01-17,S002,Tablet,6,1800,No
2024-01-18,S001,Laptop,7,7000,Yes
2024-01-18,S001,Phone,10,5000,Yes
2024-01-18,S001,Tablet,6,1800,Yes
2024-01-18,S002,Laptop,5,5000,Yes
2024-01-18,S002,Phone,16,8000,Yes
2024-01-18,S002,Tablet,8,2400,Yes
2024-01-19,S001,Laptop,4,4000,No
2024-01-19,S001,Phone,12,6000,No
2024-01-19,S001,Tablet,7,2100,No
2024-01-19,S002,Laptop,3,3000,No
2024-01-19,S002,Phone,12,6000,No
2024-01-19,S002,Tablet,8,2400,No
2024-01-20,S001,Laptop,6,6000,No
2024-01-20,S001,Phone,8,4000,No
2024-01-20,S001,Tablet,6,1800,No
2024-01-20,S002,Laptop,8,8000,No
2024-01-20,S002,Phone,9,4500,No
2024-01-20,S002,Tablet,8,2400,No
2024-01-21,S001,Laptop,3,3000,No
2024-01-21,S001,Phone,9,4500,No
2024-01-21,S001,Tablet,5,1500,No
2024-01-21,S002,Laptop,8,8000,No
2024-01-21,S002,Phone,15,7500,No
2024-01-21,S002,Tablet,7,2100,No
2024-01-22,S001,Laptop,1,1000,No
2024-01-22,S001,Phone,15,7500,No
2024-01-22,S001,Tablet,5,1500,No
2024-01-22,S002,Laptop,11,11000,No
2024-01-22,S002,Phone,4,2000,No
2024-01-22,S002,Tablet,4,1200,No
2024-01-23,S001,Laptop,3,3000,No
2024-01-23,S001,Phone,8,4000,No
2024-01-23,S001,Tablet,8,2400,No
2024-01-23,S002,Laptop,6,6000,No
2024-01-23,S002,Phone,12,6000,No
2024-01-23,S002,Tablet,12,3600,No
2024-01-24,S001,Laptop,2,2000,No
2024-01-24,S001,Phone,14,7000,No
2024-01-24,S001,Tablet,6,1800,No
2024-01-24,S002,Laptop,1,1000,No
2024-01-24,S002,Phone,5,2500,No
2024-01-24,S002,Tablet,7,2100,No
2024-01-25,S001,Laptop,7,7000,No
2024-01-25,S001,Phone,11,5500,No
2024-01-25,S001,Tablet,11,3300,No
2024-01-25,S002,Laptop,6,6000,No
2024-01-25,S002,Phone,11,5500,No
2024-01-25,S002,Tablet,5,1500,No
2024-01-26,S001,Laptop,5,5000,Yes
2024-01-26,S001,Phone,22,11000,Yes
2024-01-26,S001,Tablet,7,2100,Yes
2024-01-26,S002,Laptop,6,6000,Yes
2024-01-26,S002,Phone,24,12000,Yes
2024-01-26,S002,Tablet,3,900,Yes
2024-01-27,S001,Laptop,7,7000,Yes
2024-01-27,S001,Phone,20,10000,Yes
2024-01-27,S001,Tablet,6,1800,Yes
2024-01-27,S002,Laptop,4,4000,Yes
2024-01-27,S002,Phone,8,4000,Yes
2024-01-27,S002,Tablet,6,1800,Yes
2024-01-28,S001,Laptop,10,10000,No
2024-01-28,S001,Phone,15,7500,No
2024-01-28,S001,Tablet,12,3600,No
2024-01-28,S002,Laptop,6,6000,No
2024-01-28,S002,Phone,11,5500,No
2024-01-28,S002,Tablet,10,3000,No
2024-01-29,S001,Laptop,3,3000,No
2024-01-29,S001,Phone,16,8000,No
2024-01-29,S001,Tablet,5,1500,No
2024-01-29,S002,Laptop,6,6000,No
2024-01-29,S002,Phone,17,8500,No
2024-01-29,S002,Tablet,2,600,No
2024-01-30,S001,Laptop,3,3000,No
2024-01-30,S001,Phone,11,5500,No
2024-01-30,S001,Tablet,2,600,No
2024-01-30,S002,Laptop,6,6000,No
2024-01-30,S002,Phone,16,8000,No
2024-01-30,S002,Tablet,8,2400,No
2024-01-31,S001,Laptop,5,5000,Yes
2024-01-31,S001,Phone,22,11000,Yes
2024-01-31,S001,Tablet,9,2700,Yes
2024-01-31,S002,Laptop,3,3000,Yes
2024-01-31,S002,Phone,14,7000,Yes
2024-01-31,S002,Tablet,4,1200,Yes
2024-02-01,S001,Laptop,2,2000,No
2024-02-01,S001,Phone,7,3500,No
2024-02-01,S001,Tablet,11,3300,No
2024-02-01,S002,Laptop,6,6000,No
2024-02-01,S002,Phone,11,5500,No
2024-02-01,S002,Tablet,5,1500,No
2024-02-02,S001,Laptop,2,2000,No
2024-02-02,S001,Phone,9,4500,No
2024-02-02,S001,Tablet,7,2100,No
2024-02-02,S002,Laptop,5,5000,No
2024-02-02,S002,Phone,9,4500,No
2024-02-02,S002,Tablet,12,3600,No
2024-02-03,S001,Laptop,9,9000,No
2024-02-03,S001,Phone,12,6000,No
2024-02-03,S001,Tablet,9,2700,No
2024-02-03,S002,Laptop,10,10000,No
2024-02-03,S002,Phone,6,3000,No
2024-02-03,S002,Tablet,10,3000,No
2024-02-04,S001,Laptop,6,6000,No
2024-02-04,S001,Phone,5,2500,No
2024-02-04,S001,Tablet,8,2400,No
2024-02-04,S002,Laptop,6,6000,No
2024-02-04,S002,Phone,10,5000,No
2024-02-04,S002,Tablet,10,3000,No
2024-02-05,S001,Laptop,7,7000,No
2024-02-05,S001,Phone,13,6500,No
2024-02-05,S001,Tablet,11,3300,No
2024-02-05,S002,Laptop,8,8000,No
2024-02-05,S002,Phone,11,5500,No
2024-02-05,S002,Tablet,8,2400,No
2024-02-06,S001,Laptop,5,5000,No
2024-02-06,S001,Phone,14,7000,No
2024-02-06,S001,Tablet,4,1200,No
2024-02-06,S002,Laptop,2,2000,No
2024-02-06,S002,Phone,11,5500,No
2024-02-06,S002,Tablet,7,2100,No
2024-02-07,S001,Laptop,6,6000,No
2024-02-07,S001,Phone,7,3500,No
2024-02-07,S001,Tablet,9,2700,No
2024-02-07,S002,Laptop,2,2000,No
2024-02-07,S002,Phone,8,4000,No
2024-02-07,S002,Tablet,9,2700,No
2024-02-08,S001,Laptop,5,5000,No
2024-02-08,S001,Phone,12,6000,No
2024-02-08,S001,Tablet,3,900,No
2024-02-08,S002,Laptop,8,8000,No
2024-02-08,S002,Phone,5,2500,No
2024-02-08,S002,Tablet,8,2400,No
2024-02-09,S001,Laptop,6,6000,Yes
2024-02-09,S001,Phone,18,9000,Yes
2024-02-09,S001,Tablet,5,1500,Yes
2024-02-09,S002,Laptop,7,7000,Yes
2024-02-09,S002,Phone,18,9000,Yes
2024-02-09,S002,Tablet,5,1500,Yes
2024-02-10,S001,Laptop,9,9000,No
2024-02-10,S001,Phone,6,3000,No
2024-02-10,S001,Tablet,8,2400,No
2024-02-10,S002,Laptop,7,7000,No
2024-02-10,S002,Phone,5,2500,No
2024-02-10,S002,Tablet,6,1800,No
2024-02-11,S001,Laptop,6,6000,No
2024-02-11,S001,Phone,11,5500,No
2024-02-11,S001,Tablet,2,600,No
2024-02-11,S002,Laptop,7,7000,No
2024-02-11,S002,Phone,5,2500,No
2024-02-11,S002,Tablet,9,2700,No
2024-02-12,S001,Laptop,5,5000,No
2024-02-12,S001,Phone,5,2500,No
2024-02-12,S001,Tablet,4,1200,No
2024-02-12,S002,Laptop,1,1000,No
2024-02-12,S002,Phone,14,7000,No
2024-02-12,S002,Tablet,15,4500,No
2024-02-13,S001,Laptop,3,3000,No
2024-02-13,S001,Phone,18,9000,No
2024-02-13,S001,Tablet,8,2400,No
2024-02-13,S002,Laptop,5,5000,No
2024-02-13,S002,Phone,8,4000,No
2024-02-13,S002,Tablet,6,1800,No
2024-02-14,S001,Laptop,4,4000,No
2024-02-14,S001,Phone,9,4500,No
2024-02-14,S001,Tablet,6,1800,No
2024-02-14,S002,Laptop,4,4000,No
2024-02-14,S002,Phone,6,3000,No
2024-02-14,S002,Tablet,7,2100,No
2024-02-15,S001,Laptop,4,4000,Yes
2024-02-15,S001,Phone,26,13000,Yes
2024-02-15,S001,Tablet,5,1500,Yes
2024-02-15,S002,Laptop,2,2000,Yes
2024-02-15,S002,Phone,14,7000,Yes
2024-02-15,S002,Tablet,6,1800,Yes
2024-02-16,S001,Laptop,7,7000,No
2024-02-16,S001,Phone,9,4500,No
2024-02-16,S001,Tablet,1,300,No
2024-02-16,S002,Laptop,6,6000,No
2024-02-16,S002,Phone,12,6000,No
2024-02-16,S002,Tablet,10,3000,No
2024-02-17,S001,Laptop,5,5000,No
2024-02-17,S001,Phone,8,4000,No
2024-02-17,S001,Tablet,14,4200,No
2024-02-17,S002,Laptop,4,4000,No
2024-02-17,S002,Phone,13,6500,No
2024-02-17,S002,Tablet,7,2100,No
2024-02-18,S001,Laptop,6,6000,Yes
2024-02-18,S001,Phone,22,11000,Yes
2024-02-18,S001,Tablet,9,2700,Yes
2024-02-18,S002,Laptop,2,2000,Yes
2024-02-18,S002,Phone,10,5000,Yes
2024-02-18,S002,Tablet,12,3600,Yes
2024-02-19,S001,Laptop,6,6000,No
2024-02-19,S001,Phone,12,6000,No
2024-02-19,S001,Tablet,3,900,No
2024-02-19,S002,Laptop,3,3000,No
2024-02-19,S002,Phone,4,2000,No
2024-02-19,S002,Tablet,7,2100,No


The following cell writes the RAG product catalog file, `data/rag/product_catalog.md`.

In [ ]:
%%writefile data/rag/product_catalog.md
# Product Catalog: Smartphones, Laptops, and Tablets

## Smartphones

The Veltrix Solis Z9 is a flagship device in the premium smartphone segment. It builds on a decade of design iterations that prioritize screen-to-body ratio, minimal bezels, and high refresh rate displays. The 6.7-inch AMOLED panel with 120Hz refresh rate delivers immersive visual experiences, whether in gaming, video streaming, or augmented reality applications. The display's GorillaGlass Fusion coating provides scratch resistance and durability, and the thin form factor is engineered using a titanium-aluminum alloy chassis to reduce weight without compromising rigidity.

Internally, the Solis Z9 is powered by the OrionEdge V14 chipset, a 4nm process SoC designed for high-efficiency workloads. Its AI accelerator module handles on-device tasks such as voice transcription, camera optimization, and intelligent background app management. The inclusion of 12GB LPDDR5 RAM and a 256GB UFS 3.1 storage system allows for seamless multitasking, instant app launching, and rapid data access. The device supports eSIM and dual physical SIM configurations, catering to global travelers and hybrid network users.

Photography and videography are central to the Solis Z9 experience. The triple-camera system incorporates a periscope-style 8MP telephoto lens with 5x optical zoom, a 12MP ultra-wide sensor with macro capabilities, and a 64MP main sensor featuring optical image stabilization (OIS) and phase detection autofocus (PDAF). Night mode and HDRX+ processing enable high-fidelity image capture in challenging lighting conditions.

Software-wise, the device ships with LunOS 15, a lightweight Android fork optimized for modular updates and privacy compliance. The system supports secure containers for work profiles and AI-powered notifications that summarize app alerts across channels. Facial unlock is augmented by a 3D IR depth sensor, providing reliable biometric security alongside the ultrasonic in-display fingerprint scanner.

The Solis Z9 is a culmination of over a decade of design experimentation in mobile form factors, ranging from curved-edge screens to under-display camera arrays. Its balance of performance, battery efficiency, and user-centric software makes it an ideal daily driver for content creators, mobile gamers, and enterprise users.

## Laptops

The Cryon Vanta 16X represents the latest evolution of portable computing power tailored for professional-grade workloads.

The Vanta 16X features a unibody chassis milled from aircraft-grade aluminum using CNC machining. The thermal design integrates vapor chamber cooling and dual-fan exhaust architecture to support sustained performance under high computational loads. The 16-inch 4K UHD display is color-calibrated at the factory and supports HDR10+, making it suitable for cinematic video editing and high-fidelity CAD modeling.

Powering the device is Intel's Core i9-13900H processor, which includes 14 cores with a hybrid architecture combining performance and efficiency cores. This allows the system to dynamically balance power consumption and raw speed based on active workloads. The dedicated Zephira RTX 4700G GPU features 8GB of GDDR6 VRAM and is optimized for CUDA and Tensor Core operations, enabling applications in real-time ray tracing, AI inference, and 3D rendering.

The Vanta 16X includes a 2TB PCIe Gen 4 NVMe SSD, delivering sequential read/write speeds above 7GB/s, and 32GB of high-bandwidth DDR5 RAM. The machine supports hardware-accelerated virtualization and dual-booting, and ships with VireoOS Pro pre-installed, with official drivers available for Fedora, Ubuntu LTS, and NebulaOS.

Input options are expansive. The keyboard features per-key RGB lighting and programmable macros, while the haptic touchpad supports multi-gesture navigation and palm rejection. Port variety includes dual Thunderbolt 4 ports, a full-size SD Express card reader, HDMI 2.1, 2.5G Ethernet, three USB-A 3.2 ports, and a 3.5mm TRRS audio jack. A fingerprint reader is embedded in the power button and supports biometric logins via Windows Hello.

The history of the Cryon laptop line dates back to the early 2010s, when the company launched its first ultrabook aimed at mobile developers. Since then, successive generations have introduced carbon fiber lids, modular SSD bays, and convertible form factors. The Vanta 16X continues this tradition by integrating a customizable BIOS, a modular fan assembly, and a trackpad optimized for creative software like Blender and Adobe Creative Suite.

Designed for software engineers, data scientists, film editors, and 3D artists, the Cryon Vanta 16X is a workstation-class laptop in a portable shell.

## Tablets

The Nebulyn Ark S12 Ultra reflects the current apex of tablet technology, combining high-end hardware with software environments tailored for productivity and creativity.

The Ark S12 Ultra is built around a 12.9-inch OLED display that supports 144Hz refresh rate and HDR10+ dynamic range. With a resolution of 2800 x 1752 pixels and a contrast ratio of 1,000,000:1, the screen delivers vibrant color reproduction ideal for design and media consumption. The display supports true tone adaptation and low blue-light filtering for prolonged use.

Internally, the tablet uses Qualcomm's Snapdragon 8 Gen 3 SoC, which includes an Adreno 750 GPU and an NPU for on-device AI tasks. The device ships with 16GB LPDDR5X RAM and 512GB of storage with support for NVMe expansion via a proprietary magnetic dock. The 11200mAh battery enables up to 15 hours of typical use and recharges to 80 percent in 45 minutes via 45W USB-C PD.

The Ark's history traces back to the original Nebulyn Tab, which launched in 2014 as an e-reader and video streaming device. Since then, the line has evolved through multiple iterations that introduced stylus support, high-refresh screens, and multi-window desktop modes. The current model supports NebulynVerse, a DeX-like environment that allows external display mirroring and full multitasking with overlapping windows and keyboard shortcuts.

Input capabilities are central to the Ark S12 Ultra’s appeal. The Pluma Stylus 3 features magnetic charging, 4096 pressure levels, and tilt detection. It integrates haptic feedback to simulate traditional pen strokes and brush textures. The device also supports a SnapCover keyboard that includes a trackpad and programmable shortcut keys. With the stylus and keyboard, users can effectively transform the tablet into a mobile workstation or digital sketchbook.

Camera hardware includes a 13MP main sensor and a 12MP ultra-wide front camera with center-stage tracking and biometric unlock. Microphone arrays with beamforming enable studio-quality call audio. Connectivity includes Wi-Fi 7, Bluetooth 5.3, and optional LTE/5G with eSIM.

Software support is robust. The device runs NebulynOS 6.0, based on Android 14L, and supports app sandboxing, multi-user profiles, and remote device management. Integration with cloud services, including SketchNimbus and ThoughtSpace, allows for real-time collaboration and syncing of content across devices.

This tablet is targeted at professionals who require a balance between media consumption, creativity, and light productivity. Typical users include architects, consultants, university students, and UX designers.

## Comparative Summary

Each of these devices—the Veltrix Solis Z9, Cryon Vanta 16X, and Nebulyn Ark S12 Ultra—represents a best-in-class interpretation of its category. The Solis Z9 excels in mobile photography and everyday communication. The Vanta 16X is tailored for high-performance applications such as video production and AI prototyping. The Ark S12 Ultra provides a canvas for creativity, note-taking, and hybrid productivity use cases.

## Historical Trends and Design Evolution

Design across all three categories is converging toward modularity, longevity, and environmental sustainability. Recycled materials, reparability scores, and software longevity are becoming integral to brand reputation and product longevity. Future iterations are expected to feature tighter integration with wearable devices, ambient AI experiences, and cross-device workflows.

<a id="installing-nat"></a>
## 0.4) Installing NeMo Agent Toolkit

NeMo Agent Toolkit can be installed through the PyPI `nvidia-nat` package, the recommended way to install `nvidia-nat` is through `uv`.

First, we will install `uv` which offers parallel downloads and faster dependency resolution.

In [ ]:
!pip install uv

There are several optional subpackages available for `nvidia-nat`. For this example, we will rely on two subpackages:
* The `langchain` subpackage contains useful components for integrating and running within [LangChain](https://python.langchain.com/docs/introduction/).
* The `llama-index` subpackage contains useful components for integrating and running within [LlamaIndex](https://developers.llamaindex.ai/python/framework/).

In [ ]:
%%bash
uv pip show -q "nvidia-nat-langchain"
nat_langchain_installed=$?
uv pip show -q "nvidia-nat-llama-index"
nat_llama_index_installed=$?
uv pip show -q "matplotlib"
matplotlib_installed=$?
if [[ ${nat_langchain_installed} -ne 0 || ${nat_llama_index_installed} -ne 0 ]]; then
    uv pip install "nvidia-nat[langchain,llama-index]"
else
    echo "nvidia-nat[langchain,llama-index] is already installed"
fi

if [[ ${matplotlib_installed} -ne 0 ]]; then
    uv pip install "matplotlib~=3.9"
else
    echo "matplotlib is already installed"
fi

<a id="creating-workflow"></a>
# 1.0) Defining Tools for the Multi-Agent Workflow

As explained in detail in previous notebooks in this series, we can use the `nat workflow create` sub-command to create the necessary directory structure for a new agent.

Within this directory we can define all of the functions that we want to be available to the agent at runtime. In this notebook specifically we are going to extend on the tool calling agent demonstrated in the previous notebook and add a sub-agent to the available tools. We will also make the sub-agent 'discoverable' by the orchestrator by defining it in the workflow's `register.py` file.

In [ ]:
!nat workflow create retail_sales_agent_nb5

In the next cells we are going to redefine tool calls explained in the previous notebook.

<a id="product-sales-tool"></a>
## 1.1) Total Product Sales Tool

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/total_product_sales_data_tool.py
from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig


class GetTotalProductSalesDataConfig(FunctionBaseConfig, name="get_total_product_sales_data"):
    """Get total sales data by product."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=GetTotalProductSalesDataConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def get_total_product_sales_data_function(config: GetTotalProductSalesDataConfig, _builder: Builder):
    """Get total sales data for a specific product."""
    import pandas as pd

    df = pd.read_csv(config.data_path)

    async def _get_total_product_sales_data(product_name: str) -> str:
        """
        Retrieve total sales data for a specific product.

        Args:
            product_name: Name of the product

        Returns:
            String message containing total sales data
        """
        df['Product'] = df["Product"].apply(lambda x: x.lower())
        revenue = df[df['Product'] == product_name]['Revenue'].sum()
        units_sold = df[df['Product'] == product_name]['UnitsSold'].sum()

        return f"Revenue for {product_name} are {revenue} and total units sold are {units_sold}"

    yield FunctionInfo.from_fn(
        _get_total_product_sales_data,
        description=_get_total_product_sales_data.__doc__)

<a id="sales-per-day-tool"></a>
## 1.2) Sales Per Day Tool

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/sales_per_day_tool.py
from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig


class GetSalesPerDayConfig(FunctionBaseConfig, name="get_sales_per_day"):
    """Get total sales across all products per day."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=GetSalesPerDayConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def sales_per_day_function(config: GetSalesPerDayConfig, builder: Builder):
    """Get total sales across all products per day."""
    import pandas as pd

    df = pd.read_csv(config.data_path)
    df['Product'] = df["Product"].apply(lambda x: x.lower())

    async def _get_sales_per_day(date: str, product: str) -> str:
        """
        Calculate total sales data across all products for a specific date.

        Args:
            date: Date in YYYY-MM-DD format
            product: Product name

        Returns:
            String message with the total sales for the day
        """
        if date == "None":
            return "Please provide a date in YYYY-MM-DD format."
        total_revenue = df[(df['Date'] == date) & (df['Product'] == product)]['Revenue'].sum()
        total_units_sold = df[(df['Date'] == date) & (df['Product'] == product)]['UnitsSold'].sum()

        return f"Total revenue for {date} is {total_revenue} and total units sold is {total_units_sold}"

    yield FunctionInfo.from_fn(
        _get_sales_per_day,
        description=_get_sales_per_day.__doc__)

<a id="detect-outliers-tool"></a>
## 1.3) Detect Outliers Tool

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/detect_outliers_tool.py
from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig


class DetectOutliersIQRConfig(FunctionBaseConfig, name="detect_outliers_iqr"):
    """Detect outliers in sales data using IQR method."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=DetectOutliersIQRConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def detect_outliers_iqr_function(config: DetectOutliersIQRConfig, _builder: Builder):
    """Detect outliers in sales data using the Interquartile Range (IQR) method."""
    import pandas as pd

    df = pd.read_csv(config.data_path)

    async def _detect_outliers_iqr(metric: str) -> str:
        """
        Detect outliers in retail data using the IQR method.

        Args:
            metric: Specific metric to check for outliers

        Returns:
            Dictionary containing outlier analysis results
        """
        if metric == "None":
            column = "Revenue"
        else:
            column = metric

        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        outliers = df[(df[column] < q1 - 1.5 * iqr) | (df[column] > q3 + 1.5 * iqr)]

        return f"Outliers in {column} are {outliers.to_dict('records')}"

    yield FunctionInfo.from_fn(
        _detect_outliers_iqr,
        description=_detect_outliers_iqr.__doc__)


<a id="technical-specs-tool"></a>
## 1.4) Technical Specs Retrieval Tool

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/retail_sales_rag_tool.py
import logging
import os

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import EmbedderRef
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)


class LlamaIndexRAGConfig(FunctionBaseConfig, name="retail_sales_rag"):

    llm_name: LLMRef = Field(description="The name of the LLM to use for the RAG engine.")
    embedder_name: EmbedderRef = Field(description="The name of the embedder to use for the RAG engine.")
    data_dir: str = Field(description="The directory containing the data to use for the RAG engine.")
    description: str = Field(description="A description of the knowledge included in the RAG system.")
    collection_name: str = Field(default="context", description="The name of the collection to use for the RAG engine.")


def _walk_directory(root: str):
    for root, dirs, files in os.walk(root):
        for file_name in files:
            yield os.path.join(root, file_name)


@register_function(config_type=LlamaIndexRAGConfig, framework_wrappers=[LLMFrameworkEnum.LLAMA_INDEX])
async def retail_sales_rag_tool(config: LlamaIndexRAGConfig, builder: Builder):
    from llama_index.core import Settings
    from llama_index.core import SimpleDirectoryReader
    from llama_index.core import StorageContext
    from llama_index.core import VectorStoreIndex
    from llama_index.core.node_parser import SentenceSplitter

    llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LLAMA_INDEX)
    embedder = await builder.get_embedder(config.embedder_name, wrapper_type=LLMFrameworkEnum.LLAMA_INDEX)

    Settings.embed_model = embedder
    Settings.llm = llm

    files = list(_walk_directory(config.data_dir))
    docs = SimpleDirectoryReader(input_files=files).load_data()
    logger.info("Loaded %s documents from %s", len(docs), config.data_dir)

    parser = SentenceSplitter(
        chunk_size=400,
        chunk_overlap=20,
        separator=" ",
    )
    nodes = parser.get_nodes_from_documents(docs)

    index = VectorStoreIndex(nodes)

    query_engine = index.as_query_engine(similarity_top_k=3, )

    async def _arun(inputs: str) -> str:
        """
        Search product catalog for information about tablets, laptops, and smartphones
        Args:
            inputs: user query about product specifications
        """
        try:
            response = query_engine.query(inputs)
            return str(response.response)

        except Exception as e:
            logger.error("RAG query failed: %s", e)
            return f"Sorry, I couldn't retrieve information about that product. Error: {str(e)}"

    yield FunctionInfo.from_fn(_arun, description=config.description)

<a id="plotting-tools"></a>
## 1.5) Data Analysis/Plotting Tools

This is a new set of tools that will be registered to the data analysis and plotting agent. This set of tools allows the registered agent to plot the results of upstream data analysis tasks.

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/data_visualization_tools.py
from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig


class PlotSalesTrendForStoresConfig(FunctionBaseConfig, name="plot_sales_trend_for_stores"):
    """Plot sales trend for a specific store."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=PlotSalesTrendForStoresConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def plot_sales_trend_for_stores_function(config: PlotSalesTrendForStoresConfig, _builder: Builder):
    """Create a visualization of sales trends over time."""
    import matplotlib.pyplot as plt
    import pandas as pd

    df = pd.read_csv(config.data_path)

    async def _plot_sales_trend_for_stores(store_id: str) -> str:
        if store_id not in df["StoreID"].unique():
            data = df
            title = "Sales Trend for All Stores"
        else:
            data = df[df["StoreID"] == store_id]
            title = f"Sales Trend for Store {store_id}"

        plt.figure(figsize=(10, 5))
        trend = data.groupby("Date")["Revenue"].sum()
        trend.plot(title=title)
        plt.xlabel("Date")
        plt.ylabel("Revenue")
        plt.tight_layout()
        plt.savefig("sales_trend.png")

        return "Sales trend plot saved to sales_trend.png"

    yield FunctionInfo.from_fn(
        _plot_sales_trend_for_stores,
        description=(
            "This tool can be used to plot the sales trend for a specific store or all stores. "
            "It takes in a store ID creates and saves an image of a plot of the revenue trend for that store."))


class PlotAndCompareRevenueAcrossStoresConfig(FunctionBaseConfig, name="plot_and_compare_revenue_across_stores"):
    """Plot and compare revenue across stores."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=PlotAndCompareRevenueAcrossStoresConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def plot_revenue_across_stores_function(config: PlotAndCompareRevenueAcrossStoresConfig, _builder: Builder):
    """Create a visualization comparing sales trends between stores."""
    import matplotlib.pyplot as plt
    import pandas as pd

    df = pd.read_csv(config.data_path)

    async def _plot_revenue_across_stores(arg: str) -> str:
        pivot = df.pivot_table(index="Date", columns="StoreID", values="Revenue", aggfunc="sum")
        pivot.plot(figsize=(12, 6), title="Revenue Trends Across Stores")
        plt.xlabel("Date")
        plt.ylabel("Revenue")
        plt.legend(title="StoreID")
        plt.tight_layout()
        plt.savefig("revenue_across_stores.png")

        return "Revenue trends across stores plot saved to revenue_across_stores.png"

    yield FunctionInfo.from_fn(
        _plot_revenue_across_stores,
        description=(
            "This tool can be used to plot and compare the revenue trends across stores. Use this tool only if the "
            "user asks for a comparison of revenue trends across stores."
            "It takes in a single string as input (which is ignored) and creates and saves an image of a plot of the revenue trends across stores."
        ))


class PlotAverageDailyRevenueConfig(FunctionBaseConfig, name="plot_average_daily_revenue"):
    """Plot average daily revenue for stores and products."""
    data_path: str = Field(description="Path to the data file")


@register_function(config_type=PlotAverageDailyRevenueConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def plot_average_daily_revenue_function(config: PlotAverageDailyRevenueConfig, _builder: Builder):
    """Create a bar chart showing average daily revenue by day of week."""
    import matplotlib.pyplot as plt
    import pandas as pd

    df = pd.read_csv(config.data_path)

    async def _plot_average_daily_revenue(arg: str) -> str:
        daily_revenue = df.groupby(["StoreID", "Product", "Date"])["Revenue"].sum().reset_index()

        avg_daily_revenue = daily_revenue.groupby(["StoreID", "Product"])["Revenue"].mean().unstack()

        avg_daily_revenue.plot(kind="bar", figsize=(12, 6), title="Average Daily Revenue per Store by Product")
        plt.ylabel("Average Revenue")
        plt.xlabel("Store ID")
        plt.xticks(rotation=0)
        plt.legend(title="Product", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig("average_daily_revenue.png")

        return "Average daily revenue plot saved to average_daily_revenue.png"

    yield FunctionInfo.from_fn(
        _plot_average_daily_revenue,
        description=("This tool can be used to plot the average daily revenue for stores and products "
                     "It takes in a single string as input and creates and saves an image of a grouped bar chart "
                     "of the average daily revenue"))

<a id="register-tools"></a>
## 1.6) Register The Tools

In [ ]:
%%writefile -a retail_sales_agent_nb5/src/retail_sales_agent_nb5/register.py

from . import sales_per_day_tool
from . import detect_outliers_tool
from . import total_product_sales_data_tool
from . import retail_sales_rag_tool
from . import data_visualization_tools

<a id="adding-orchestrator"></a>
# 2.0) Adding an Agent Orchestrator

<a id="orchestrator-config"></a>
## 2.1) Agent Orchestrator Workflow Configuration File

Next, we introduce a new workflow configuration file. Upon first glance, this configuration file may seem complex. However, when we classify the available tools and delegate them to an agent, it will begin to make intuitive sense how sub-tasks in a workflow can be divided amongst the best fit agent.

In this multi-agent orchestration system, we divide responsibilities among specialized agents, each equipped with tools that match their domain expertise:

**1. Data Analysis Agent**
- **Tools:** `total_product_sales_data`, `sales_per_day`, `detect_outliers`
- **Justification:** This agent handles raw data processing and statistical analysis. These tools extract, aggregate, and analyze sales data, making it the expert for answering questions about sales trends, patterns, and anomalies. By isolating data analysis tasks, we ensure consistent and reliable data interpretation.

**2. Visualization Agent**
- **Tools:** `plot_total_product_sales`, `plot_sales_per_day`, `plot_average_daily_revenue`
- **Justification:** This agent specializes in creating visual representations of data. Visualization requires different expertise than raw data analysis—it involves understanding chart types, formatting, and visual communication. Separating this from data analysis allows the agent to focus on producing clear, effective visualizations without mixing concerns.

**3. Knowledge Retrieval Agent (RAG)**
- **Tools:** `retail_sales_rag`
- **Justification:** This agent accesses external knowledge bases and documentation through retrieval-augmented generation. It handles questions that require contextual information beyond the sales data itself, such as business policies, product information, or historical context. This separation ensures that knowledge retrieval doesn't interfere with computational tasks.

**4. Orchestrator Agent (Top-Level)**
- **Tools:** None (delegates to sub-agents)
- **Justification:** The orchestrator doesn't perform tasks directly but instead routes requests to the appropriate expert agent. This design pattern mirrors real-world organizational structures where a manager delegates to specialists. It enables complex workflows where multiple agents collaborate, each contributing their expertise to solve multi-faceted problems.

This architecture provides several benefits:
- **Modularity:** Each agent can be updated or replaced independently
- **Clarity:** Tool responsibilities are clearly defined and scoped
- **Scalability:** New agents and tools can be added without disrupting existing ones
- **Efficiency:** Agents only load and reason about tools relevant to their domain

> **Note:** _You will notice in the below configuration that no tools are directly called by the workflow-level agent. Instead, it delegates specifically to expert agents based on the request_

In [ ]:
%%writefile retail_sales_agent_nb5/configs/config_multi_agent.yml
llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 2048
    context_window: 32768
    api_key: $NVIDIA_API_KEY

embedders:
  nim_embedder:
    _type: nim
    model_name: nvidia/nv-embedqa-e5-v5
    truncate: END
    api_key: $NVIDIA_API_KEY

functions:
  total_product_sales_data:
    _type: get_total_product_sales_data
    data_path: data/retail_sales_data.csv
  sales_per_day:
    _type: get_sales_per_day
    data_path: data/retail_sales_data.csv
  detect_outliers:
    _type: detect_outliers_iqr
    data_path: data/retail_sales_data.csv

  data_analysis_agent:
    _type: tool_calling_agent
    tool_names:
      - total_product_sales_data
      - sales_per_day
      - detect_outliers
    llm_name: nim_llm
    max_history: 10
    max_iterations: 15
    description: |
      A helpful assistant that can answer questions about the retail sales CSV data.
      Use the tools to answer the questions.
      Input is a single string.
    verbose: false

  product_catalog_rag:
    _type: retail_sales_rag
    llm_name: nim_llm
    embedder_name: nim_embedder
    collection_name: product_catalog_rag
    data_dir: data/rag/
    description: "Search product catalog for TabZen tablet, AeroBook laptop, NovaPhone specifications"

  rag_agent:
    _type: react_agent
    llm_name: nim_llm
    tool_names: [product_catalog_rag]
    max_history: 3
    max_iterations: 5
    max_retries: 2
    description: |
      An assistant that can only answer questions about products.
      Use the product_catalog_rag tool to answer questions about products.
      Do not make up any information.
    verbose: false

  plot_sales_trend_for_stores:
    _type: plot_sales_trend_for_stores
    data_path: data/retail_sales_data.csv
  plot_and_compare_revenue_across_stores:
    _type: plot_and_compare_revenue_across_stores
    data_path: data/retail_sales_data.csv
  plot_average_daily_revenue:
    _type: plot_average_daily_revenue
    data_path: data/retail_sales_data.csv

  data_visualization_agent:
    _type: react_agent
    llm_name: nim_llm
    tool_names:
      - plot_sales_trend_for_stores
      - plot_and_compare_revenue_across_stores
      - plot_average_daily_revenue
    max_history: 10
    max_iterations: 15
    description: |
      You are a data visualization expert.
      You can only create plots and visualizations based on user requests.
      Only use available tools to generate plots.
      You cannot analyze any data.
    verbose: false
    handle_parsing_errors: true
    max_retries: 2
    retry_parsing_errors: true

workflow:
  _type: react_agent
  tool_names: [data_analysis_agent, data_visualization_agent, rag_agent]
  llm_name: nim_llm
  verbose: true
  handle_parsing_errors: true
  max_retries: 2
  system_prompt: |
    Answer the following questions as best you can.
    You may communicate and collaborate with various experts to answer the questions.

    {tools}

    You may respond in one of two formats.
    Use the following format exactly to communicate with an expert:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action (if there is no required input, include "Action Input: None")
    Observation: wait for the expert to respond, do not assume the expert's response

    ... (this Thought/Action/Action Input/Observation can repeat N times.)
    Use the following format once you have the final answer:

    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

<a id="running-orchestrator-workflow"></a>
## 2.2) Running the Workflow

Next we can run the workflow:


In [ ]:
!nat run --config_file retail_sales_agent_nb5/configs/config_multi_agent.yml \
  --input "What is the Ark S12 Ultra tablet and what are its specifications?" \
  --input "How do laptop sales compare to phone sales?" \
  --input "Plot average daily revenue"

If images were generated by tool calls you can view them by running the following code cell:

In [ ]:
from IPython.display import Image
from IPython.display import display

display(Image("./average_daily_revenue.png"))

<a id="adding-custom-agent"></a>
# 3.0) Adding a Custom Agent

Besides using inbuilt agents in the workflows, we can also create custom agents using LangGraph or any other framework and bring them into a workflow. We demonstrate this by swapping out the ReAct agent used by the data visualization expert for a custom agent that has human-in-the-loop capability. The agent will ask the user whether they would like a summary of graph content.

This exemplifies how complete agent workflows can be wrapped and used as tools by other agents, enabling complex multi-agent orchestration.

<a id="hitl-tool"></a>
## 3.1) Human-in-the-Loop (HITL) Approval Tool

The following two cells define the approval tool and its registration.

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/retail_hitl_approval.py
import logging

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.context import Context
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig
from nat.data_models.interactive import HumanPromptText
from nat.data_models.interactive import InteractionResponse

logger = logging.getLogger(__name__)


class HITLApprovalFnConfig(FunctionBaseConfig, name="retail_hitl_approval"):
    """
    This function is used to get the user's response to the prompt.
    It will return True if the user responds with 'yes', otherwise False.
    """

    prompt: str = Field(..., description="The prompt to use for the HITL function")


@register_function(config_type=HITLApprovalFnConfig)
async def hitl_approval_function(config: HITLApprovalFnConfig, builder: Builder):

    import re

    prompt = f"{config.prompt} Please confirm if you would like to proceed. Respond with 'yes' or 'no'."

    async def _arun(unused: str = "") -> bool:

        nat_context = Context.get()
        user_input_manager = nat_context.user_interaction_manager

        human_prompt_text = HumanPromptText(text=prompt, required=True, placeholder="<your response here>")
        response: InteractionResponse = await user_input_manager.prompt_user_input(human_prompt_text)
        response_str = response.content.text.lower()  # type: ignore
        selected_option = re.search(r'\b(yes)\b', response_str)

        if selected_option:
            return True
        return False

    yield FunctionInfo.from_fn(_arun,
                               description=("This function will be used to get the user's response to the prompt"))

In [ ]:
%%writefile -a retail_sales_agent_nb5/src/retail_sales_agent_nb5/register.py

from . import retail_hitl_approval

<a id="graph-summarizer-tool"></a>
## 3.2) Graph Summarizer Tool

The following two cells define the graph summarizer tool and its registration.

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/graph_summarizer_tool.py
from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig


class GraphSummarizerConfig(FunctionBaseConfig, name="graph_summarizer"):
    """Analyze and summarize chart data."""
    llm_name: LLMRef = Field(description="The name of the LLM to use for the graph summarizer.")


@register_function(config_type=GraphSummarizerConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def graph_summarizer_function(config: GraphSummarizerConfig, builder: Builder):
    """Analyze chart data and provide natural language summaries."""
    import base64

    from langchain_core.messages import HumanMessage

    llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)

    async def _graph_summarizer(image_path: str) -> str:
        """
        Analyze chart data and provide insights and summaries.

        Args:
            image_path: The path to the image to analyze

        Returns:
            String containing analysis and insights
        """

        def encode_image(image_path: str) -> str:
            with open(image_path, "rb") as image_file:
                return base64.b64encode(image_file.read()).decode('utf-8')

        base64_image = encode_image(image_path)

        # Create a multimodal message with text and image
        message = HumanMessage(
            content=[
                {"type": "text", "text": "Please summarize the key insights from this graph in natural language."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ]
        )

        # Invoke the LLM with the multimodal message
        response = await llm.ainvoke([message])
        return response.content

    yield FunctionInfo.from_fn(
        _graph_summarizer,
        description=("This tool can be used to summarize the key insights from a graph in natural language. "
                     "It takes in the path to an image and returns a summary of the key insights from the graph."))

In [ ]:
%%writefile -a retail_sales_agent_nb5/src/retail_sales_agent_nb5/register.py

from . import graph_summarizer_tool

<a id="custom-viz-agent"></a>
## 3.3) Custom Data Visualization Agent With HITL Approval

The following two cells define the custom agent and its registration

In [ ]:
%%writefile retail_sales_agent_nb5/src/retail_sales_agent_nb5/data_visualization_agent.py
import logging

from pydantic import Field

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function import Function
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import FunctionRef
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)


class DataVisualizationAgentConfig(FunctionBaseConfig, name="data_visualization_agent"):
    """
    NeMo Agent Toolkit function config for data visualization.
    """
    llm_name: LLMRef = Field(description="The name of the LLM to use")
    tool_names: list[FunctionRef] = Field(description="The names of the tools to use")
    description: str = Field(description="The description of the agent.")
    prompt: str = Field(description="The prompt to use for the agent.")
    graph_summarizer_fn: FunctionRef = Field(description="The function to use for the graph summarizer.")
    hitl_approval_fn: FunctionRef = Field(description="The function to use for the hitl approval.")
    max_retries: int = Field(default=3, description="The maximum number of retries for the agent.")


@register_function(config_type=DataVisualizationAgentConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def data_visualization_agent_function(config: DataVisualizationAgentConfig, builder: Builder):
    from langchain_core.messages import AIMessage
    from langchain_core.messages import BaseMessage
    from langchain_core.messages import HumanMessage
    from langchain_core.messages import SystemMessage
    from langchain_core.messages import ToolMessage
    from langgraph.graph import StateGraph
    from langgraph.prebuilt import ToolNode
    from pydantic import BaseModel

    class AgentState(BaseModel):
        retry_count: int = 0
        messages: list[BaseMessage]
        approved: bool = True

    tools = await builder.get_tools(tool_names=config.tool_names, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
    llm = await builder.get_llm(llm_name=config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
    llm_n_tools = llm.bind_tools(tools)

    hitl_approval_fn: Function = await builder.get_function(config.hitl_approval_fn)
    graph_summarizer_fn: Function = await builder.get_function(config.graph_summarizer_fn)

    async def conditional_edge(state: AgentState):
        try:
            logger.debug("Starting the Tool Calling Conditional Edge")
            messages = state.messages
            last_message = messages[-1]
            logger.info("Last message type: %s", type(last_message))
            logger.info("Has tool_calls: %s", hasattr(last_message, 'tool_calls'))
            if hasattr(last_message, 'tool_calls'):
                logger.info("Tool calls: %s", last_message.tool_calls)

            if (hasattr(last_message, 'tool_calls') and last_message.tool_calls and len(last_message.tool_calls) > 0):
                logger.info("Routing to tools - found non-empty tool calls")
                return "tools"
            logger.info("Routing to check_hitl_approval - no tool calls to execute")
            return "check_hitl_approval"
        except Exception as ex:
            logger.error("Error in conditional_edge: %s", ex)
            if hasattr(state, 'retry_count') and state.retry_count >= config.max_retries:
                logger.warning("Max retries reached, returning without meaningful output")
                return "__end__"
            state.retry_count = getattr(state, 'retry_count', 0) + 1
            logger.warning(
                "Error in the conditional edge: %s, retrying %d times out of %d",
                ex,
                state.retry_count,
                config.max_retries,
            )
            return "data_visualization_agent"

    def approval_conditional_edge(state: AgentState):
        """Route to summarizer if user approved, otherwise end"""
        logger.info("Approval conditional edge: %s", state.approved)
        if hasattr(state, 'approved') and not state.approved:
            return "__end__"
        return "summarize"

    def data_visualization_agent(state: AgentState):
        sys_msg = SystemMessage(content=config.prompt)
        messages = state.messages

        if messages and isinstance(messages[-1], ToolMessage):
            last_tool_msg = messages[-1]
            logger.info("Processing tool result: %s", last_tool_msg.content)
            summary_content = f"I've successfully created the visualization. {last_tool_msg.content}"
            return {"messages": [AIMessage(content=summary_content)]}
        logger.info("Normal agent operation - generating response for: %s", messages[-1] if messages else 'no messages')
        return {"messages": [llm_n_tools.invoke([sys_msg] + state.messages)]}

    async def check_hitl_approval(state: AgentState):
        messages = state.messages
        last_message = messages[-1]
        logger.info("Checking hitl approval: %s", state.approved)
        logger.info("Last message type: %s", type(last_message))
        selected_option = await hitl_approval_fn.acall_invoke()
        if selected_option:
            return {"approved": True}
        return {"approved": False}

    async def summarize_graph(state: AgentState):
        """Summarize the graph using the graph summarizer function"""
        image_path = None
        for msg in state.messages:
            if hasattr(msg, 'content') and msg.content:
                content = str(msg.content)
                import re
                img_ext = r'[a-zA-Z0-9_.-]+\.(?:png|jpg|jpeg|gif|svg)'
                pattern = rf'saved to ({img_ext})|({img_ext})'
                match = re.search(pattern, content)
                if match:
                    image_path = match.group(1) or match.group(2)
                    break

        if not image_path:
            image_path = "sales_trend.png"

        logger.info("Extracted image path for summarization: %s", image_path)
        response = await graph_summarizer_fn.ainvoke(image_path)
        return {"messages": [response]}

    try:
        logger.debug("Building and compiling the Agent Graph")
        builder_graph = StateGraph(AgentState)

        builder_graph.add_node("data_visualization_agent", data_visualization_agent)
        builder_graph.add_node("tools", ToolNode(tools))
        builder_graph.add_node("check_hitl_approval", check_hitl_approval)
        builder_graph.add_node("summarize", summarize_graph)

        builder_graph.add_conditional_edges("data_visualization_agent", conditional_edge)

        builder_graph.set_entry_point("data_visualization_agent")
        builder_graph.add_edge("tools", "data_visualization_agent")

        builder_graph.add_conditional_edges("check_hitl_approval", approval_conditional_edge)

        builder_graph.add_edge("summarize", "__end__")

        agent_executor = builder_graph.compile()

        logger.info("Data Visualization Agent Graph built and compiled successfully")

    except Exception as ex:
        logger.error("Failed to build Data Visualization Agent Graph: %s", ex)
        raise

    async def _arun(user_query: str) -> str:
        """
        Visualize data based on user query.

        Args:
            user_query (str): User query to visualize data

        Returns:
            str: Visualization conclusion from the LLM agent
        """
        input_message = f"User query: {user_query}."
        response = await agent_executor.ainvoke({"messages": [HumanMessage(content=input_message)]})

        return response

    try:
        yield FunctionInfo.from_fn(_arun, description=config.description)

    except GeneratorExit:
        print("Function exited early!")
    finally:
        print("Cleaning up retail_sales_agent workflow.")

In [ ]:
%%writefile -a retail_sales_agent_nb5/src/retail_sales_agent_nb5/register.py

from . import data_visualization_agent

<a id="custom-agent-config"></a>
## 3.4) Custom Agent Workflow Configuration File

Next, we define the workflow configuration file for this custom agent.

The high-level changes include:
- switching from a ReAct agent to the custom agent with HITL
- adding additional tools (HITL, graph summarization)
- adding an OpenAI LLM for image summarization

In [ ]:
%%writefile retail_sales_agent_nb5/configs/config_multi_agent_hitl.yml
llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 2048
    context_window: 32768
    api_key: $NVIDIA_API_KEY
  summarizer_llm:
    _type: nim
    model_name: meta/llama-3.2-90b-vision-instruct
    temperature: 0.0
    api_key: $NVIDIA_API_KEY

embedders:
  nim_embedder:
    _type: nim
    model_name: nvidia/nv-embedqa-e5-v5
    truncate: END
    api_key: $NVIDIA_API_KEY

functions:
  total_product_sales_data:
    _type: get_total_product_sales_data
    data_path: data/retail_sales_data.csv
  sales_per_day:
    _type: get_sales_per_day
    data_path: data/retail_sales_data.csv
  detect_outliers:
    _type: detect_outliers_iqr
    data_path: data/retail_sales_data.csv

  data_analysis_agent:
    _type: tool_calling_agent
    tool_names:
      - total_product_sales_data
      - sales_per_day
      - detect_outliers
    llm_name: nim_llm
    max_history: 10
    max_iterations: 15
    description: |
      A helpful assistant that can answer questions about the retail sales CSV data.
      Use the tools to answer the questions.
      Input is a single string.
    verbose: false

  plot_sales_trend_for_stores:
    _type: plot_sales_trend_for_stores
    data_path: data/retail_sales_data.csv
  plot_and_compare_revenue_across_stores:
    _type: plot_and_compare_revenue_across_stores
    data_path: data/retail_sales_data.csv
  plot_average_daily_revenue:
    _type: plot_average_daily_revenue
    data_path: data/retail_sales_data.csv

  retail_hitl_approval:
    _type: retail_hitl_approval
    prompt: |
      Do you want to summarize the created graph content?
  graph_summarizer:
    _type: graph_summarizer
    llm_name: summarizer_llm

  data_visualization_agent:
    _type: data_visualization_agent
    llm_name: nim_llm
    tool_names:
      - plot_sales_trend_for_stores
      - plot_and_compare_revenue_across_stores
      - plot_average_daily_revenue
    graph_summarizer_fn: graph_summarizer
    hitl_approval_fn: retail_hitl_approval
    prompt: |
      You are a data visualization expert.
      Your task is to create plots and visualizations based on user requests.
      Use available tools to analyze data and generate plots.
    description: |
      This is a data visualization agent that should be called if the user asks for a visualization or plot of the data.
      It has access to the following tools:
      - plot_sales_trend_for_stores: This tool can be used to plot the sales trend for a specific store or all stores.
      - plot_and_compare_revenue_across_stores: This tool can be used to plot and compare the revenue trends across stores. Use this tool only if the user asks for a comparison of revenue trends across stores.
      - plot_average_daily_revenue: This tool can be used to plot the average daily revenue for stores and products.
      The agent will use the available tools to analyze data and generate plots.
      The agent will also use the graph_summarizer tool to summarize the graph data.
      The agent will also use the retail_hitl_approval to ask the user whether they would like a summary of the graph data.

  product_catalog_rag:
    _type: retail_sales_rag
    llm_name: nim_llm
    embedder_name: nim_embedder
    collection_name: product_catalog_rag
    data_dir: data/rag/
    description: "Search product catalog for TabZen tablet, AeroBook laptop, NovaPhone specifications"

  rag_agent:
    _type: react_agent
    llm_name: nim_llm
    tool_names:
      - product_catalog_rag
    max_history: 3
    max_iterations: 5
    max_retries: 2
    retry_parsing_errors: true
    description: |
      An assistant that can answer questions about products.
      Use product_catalog_rag to answer questions about products.
      Do not make up information.
    verbose: true


workflow:
  _type: react_agent
  tool_names:
    - data_analysis_agent
    - data_visualization_agent
    - rag_agent
  llm_name: summarizer_llm
  verbose: true
  handle_parsing_errors: true
  max_retries: 2
  system_prompt: |
    Answer the following questions as best you can. You may communicate and collaborate with various experts to answer the questions:

    {tools}

    If the user responds "no" to a request to continue, you should end the conversation.

    You may respond in one of two formats.
    Use the following format exactly to communicate with an expert:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action (if there is no required input, include "Action Input: None")
    Observation: wait for the expert to respond, do not assume the expert's response

    ... (this Thought/Action/Action Input/Observation can repeat N times.)
    Use the following format once you have the final answer:

    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

<a id="running-custom-workflow"></a>
## 3.5) Running the Workflow

The HITL tool will prompt the user for input when it is called. This works when running in an interactive terminal, but in a notebook environment like this one, the prompt may not function as expected, so we will simulate user input by echoing "yes" into the command. This can be replaced with "no" to simulate a negative response, or run directly in a terminal for interactive input.

In [ ]:
! echo "yes" | nat run --config_file retail_sales_agent_nb5/configs/config_multi_agent_hitl.yml \
    --input "Plot average daily revenue"

This concludes this example. We've gone through several examples of integrating tools and custom agents in NeMo Agent Toolkit.

<a id="next-steps"></a>
# 4.0) Next Steps

The next notebook in this series is `observability_evaluation_and_profiling.ipynb` where we will demonstrate how to equip your agent with profiling instrumentation for improved observability over your platform.